## CS431/631 Big Data Infrastructure
---

---
#### Overview of Pointwise Mutual Information

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

The "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.


Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt -P C:\Users\shah_\Desktop
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

In [34]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
import itertools

tokens = []
pair_list= []

FILE = "Shakespeare.txt"
TEST_FILE = "tmp.txt"
# Now, let's tokenize Shakespeare's plays
with open(TEST_FILE) as f:
    for line in f:
        # tokenize, one line at a time
        t = list(set(simple_tokenize(line)))
        pair_list += list(itertools.permutations(t, 2))
        tokens+=t

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

# get unique tokens
unique_tokens = list(set(tokens))
pair_list_set = list(set(pair_list))

print("Distinct Tokens:", len(unique_tokens))
print("Distinct Pairs:", len(pair_list_set))

Distinct Tokens: 3
Distinct Pairs: 6
Distinct Pairs: 6


[('self', 'thy'),
 ('self', 'foe'),
 ('thy', 'self'),
 ('thy', 'foe'),
 ('foe', 'self'),
 ('foe', 'thy')]

In [168]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log10
import numpy as np
from collections import defaultdict

def get_pmi_analysis(file, token_1, token_2=None, threshold=None):
    # Now, let's tokenize Shakespeare's plays
    line_count = 0
    token_1_count = 0
    token_2_count = 0
    tokens_count = 0
    tokens = []

    if token_2:

        with open(file) as f:
            for line in f:
                # tokenize, one line at a time
                t = simple_tokenize(line)

                # Count
                if token_1 in t:
                    token_1_count += 1

                if token_2 in t:
                    token_2_count += 1

                if token_1 in t and token_2 in t:
                    tokens_count += 1

                line_count += 1
                tokens += set(t)
        
        # Calculate Probability
        p_1 = token_1_count / line_count
        p_2 = token_2_count / line_count
        p_1_2 = tokens_count / line_count

        # Calculate PMI
        try:
            pmi = log10(p_1_2 / (p_1*p_2))
        except Exception as e:
            pmi = np.NINF

        return tokens_count, pmi 

    else:
        line_count = 0
        kv_count_dict = defaultdict(int)

        with open(file) as f:
            for line in f:

                line_count+=1
                # tokenize, one line at a time
                t = list(set(simple_tokenize(line)))

                # Add counts to the dictionary
                if token_1 in t:
                    kv_count_dict[token_1] += 1
                    t.remove(token_1)
                
                    for other in t:
                        kv_count_dict[token_1 + "," + other] += 1 # "," indicates combined events

                for other in t:
                        kv_count_dict[other] += 1
        
        # Get a list of relevant other tokens
        combined_dict = {k:v for k, v in kv_count_dict.items() if "," in k}
        other_relavant_tokens = [k.split(',')[1] for k, v in combined_dict.items() if v>=threshold]

        # Calculate probability of independent and combined events
        p_dict = {k:v/line_count for k, v in kv_count_dict.items()}

        # Calculate PMI for relevant tokens satisfying threshold requirement
        pmi_dict = defaultdict(int)

        for other in other_relavant_tokens:
            pmi_dict[other] = [log10(p_dict[token_1 + "," + other] / (p_dict[token_1]*p_dict[other])),
                                kv_count_dict[token_1 + "," + other]]
            
        # Sort and Filter
        pmi = sorted(pmi_dict.items(), key=lambda kv: kv[1][0], reverse=True)
        
        if len(pmi)>5:
            pmi = pmi[:5]

        return kv_count_dict[token_1], pmi

###################################################################################################################

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.

        token_count, pmi = get_pmi_analysis('Shakespeare.txt', q_tokens[0], threshold=threshold)
        print("  n({0}) = {1}".format(q_tokens[0], token_count))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))

        for output in pmi:
            print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(output[0], q_tokens[0], output[1][1], output[1][0]))    
    elif len(q_tokens) == 2:
        # Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        tokens_count, pmi = get_pmi_analysis('Shakespeare.txt', q_tokens[0], q_tokens[1])
        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],tokens_count))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],pmi))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!") 

# I have used log10 instead of the default log which is actually the natural log

  n(perfect) = 54
  high PMI tokens with respect to perfect (threshold: 10):
    n(in,perfect) = 11,  PMI(in,perfect) = 0.3711070042455013
    n(i,perfect) = 12,  PMI(i,perfect) = 0.16393283689508215
    n(and,perfect) = 14,  PMI(and,perfect) = 0.11071571970717087
    n(the,perfect) = 10,  PMI(the,perfect) = -0.030012871217669154


## Note

The program's execution will become slower and slower as the file size increases since more lines will have to be read and/or the size of a single line may increase as well depending on the input. In both the situations the program will take more time.

Furthermore, there will come a point where the program will crash since all the variables are stored in RAM of the local computer which is limited in size. For example, with a bigger file size, the size of my count dictionary `kv_count_dict` (among others) will increase signficantly such that the RAM won't be able to hold its value and eventually the program will crash.